In [ ]:
import numpy as np
import pandas as pd
import os
import xgboost as xgb
import matplotlib.pyplot as plt

from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import LabelEncoder, StandardScaler

import nibabel as nib
from scipy.ndimage import center_of_mass
from nilearn.plotting import plot_connectome

# LOAD DATA 
X_fc = np.load("cc200_fc_X_children.npy")
X_fc = np.nan_to_num(X_fc, nan=0.0)
y = np.load("cc200_fc_y_children.npy")

pheno = pd.read_csv("Phenotypic_V1_0b_preprocessed1.csv")
pheno = pheno[(pheno["AGE_AT_SCAN"] < 18) & pheno["DX_GROUP"].isin([1, 2])].copy()
pheno["label"] = pheno["DX_GROUP"] - 1
pheno["SEX"]   = pheno["SEX"].map({1: 1, 2: 0})
pheno["FIQ"]   = pheno["FIQ"].fillna(pheno["FIQ"].mean())
pheno["AGE_AT_SCAN"] = pheno["AGE_AT_SCAN"].fillna(pheno["AGE_AT_SCAN"].mean())
pheno["SITE_ID"]      = LabelEncoder().fit_transform(pheno["SITE_ID"])

# Match and preprocess
subject_ids = [f.split("_rois")[0] for f in os.listdir("nyu_rois") if f.endswith(".1D")]
pheno = pheno[pheno["FILE_ID"].isin(subject_ids)].reset_index(drop=True)

X_demo = pheno[["AGE_AT_SCAN", "SEX", "FIQ", "SITE_ID"]].values
X_demo = StandardScaler().fit_transform(X_demo)

X_combined = np.hstack([X_fc[:len(X_demo)], X_demo])
y_combined = pheno["label"].values

#SelectKBest
selector = SelectKBest(score_func=f_classif, k=2500)
X_selected = selector.fit_transform(X_combined, y_combined)
mask = selector.get_support(indices=True)  # indices of selected features

# Train model
model = xgb.XGBClassifier(
    n_estimators=150,
    max_depth=5,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    eval_metric='logloss',
    random_state=42
)
model.fit(X_selected, y_combined)

# Feature importances
importances = model.feature_importances_
top_indices = np.argsort(importances)[-20:][::-1]
top_values  = importances[top_indices]

# Map feature indices to name
feature_names = []
for idx in mask[top_indices]:
    if idx < 19900:
        feature_names.append(f"Conn {idx}")
    else:
        feature_names.append(["Age","Sex","FIQ","Site"][idx - 19900])

# Plot top 20 bar chart
plt.figure(figsize=(10, 6))
plt.barh(range(len(top_values)), top_values[::-1])
plt.yticks(range(len(top_values)), feature_names[::-1])
plt.xlabel("Importance")
plt.title("Top 20 Features (XGBoost + SelectKBest)")
plt.tight_layout()
plt.savefig("Top_20_Features.png", dpi=300, bbox_inches="tight")
plt.show()

# TEXTUAL OUTPUT
print("\n Top 20 Most Important Features:")
for i, idx in enumerate(mask[top_indices]):
    if idx < 19900:
        name = f"Conn {idx}"
    else:
        name = ["Age","Sex","FIQ","Site"][idx - 19900]
    print(f"{i+1:2d}. {name:<10} | Importance: {top_values[i]:.4f}")

# Lateralization of Top 20 Connections

# Load CC200 atlas and extract centroids for each ROI label (1–200)
atlas_img  = nib.load("cc200_roi_atlas.nii.gz")
atlas_data = atlas_img.get_fdata()
roi_labels = np.unique(atlas_data)[1:].astype(int)  # skip 0 background

centroids = {}
for lbl in roi_labels:
    centroids[lbl] = center_of_mass(atlas_data == lbl)

# Build adjacency matrix for top-20 connections
n_rois  = 200
triu_i, triu_j = np.triu_indices(n_rois, k=1)
adj     = np.zeros((n_rois, n_rois))

for feat_idx, imp in zip(top_indices, top_values):
    i = triu_i[feat_idx]
    j = triu_j[feat_idx]
    adj[i, j] = imp
    adj[j, i] = imp

# Prepare list of coords in MNI space
# ROI labels in atlas are 1-based; our i,j indices are 0-based
coords = [centroids.get(l+1, (0,0,0)) for l in range(n_rois)]

# Plot connectome highlighting top connections
from nilearn.plotting import plot_connectome

plot_connectome(
    adjacency_matrix=adj,
    node_coords=coords,
    edge_kwargs={'linewidth': 2.0, 'alpha': 0.7},
    title="Top-20 Connections Lateralization",
    node_size=50
)
plt.savefig("Figure5_3_lateralization.png", dpi=300, bbox_inches="tight")
plt.show()